In [27]:
from openai import OpenAI
import json
# pip install openai==0.28

# Set your GPT API key
key = "sk-xxx" # replace xxx with the valid key
client = OpenAI(
    base_url="https://xiaoai.plus/v1",
    api_key=key
)


In [28]:
# Import the OpenAI library
import openai

# Restaurant data
# Restaurant data from the Appendix
restaurants = [
    {
        "name": "Casa Bella",
        "quality": "theta_H",
        "info": {
            "Cuisine Type": "Italian restaurant offering a wide range of authentic dishes.",
            "Food Quality": "Exceptional, with handmade gnocchi and wood-fired Margherita pizza featuring rich flavors and perfect execution, using fresh, locally sourced ingredients.",
            "Service": "Warm, attentive, and knowledgeable, with staff showing enthusiasm and ensuring guests feel welcomed.",
            "Ambiance": "Cozy yet sophisticated, with soft lighting and tasteful decor creating an inviting atmosphere for various occasions.",
            "Cleanliness": "Impeccable, meeting high standards throughout the establishment.",
            "Pricing/Value": "Reasonable for the quality, offering good value for the dining experience.",
        },
    },
    {
        "name": "Burger Bonanza",
        "quality": "theta_L",
        "info": {
            "Cuisine Type": "Casual dining spot focusing on burgers and fries.",
            "Food Quality": "Inconsistent and poor, with overcooked burgers lacking seasoning and soggy fries that seem pre-prepared.",
            "Service": "Disorganized and inattentive, resulting in long waits despite a half-empty dining room.",
            "Ambiance": "Lackluster, with outdated decor and uncleaned tables contributing to an unappealing environment.",
            "Cleanliness": "Subpar, with tables not properly cleaned between guests.",
            "Pricing/Value": "Low prices that do not justify the subpar experience.",
        },
    },
    {
        "name": "Sakura Sushi",
        "quality": "theta_H",
        "info": {
            "Cuisine Type": "Japanese restaurant specializing in sushi and sashimi.",
            "Food Quality": "High-quality, with precise sushi and sashimi using fresh, high-grade fish, highlighted by a creative omakase menu and flavorful cooked dishes like miso-glazed black cod.",
            "Service": "Exemplary, with thoughtful recommendations and a seamless dining experience provided by attentive servers.",
            "Ambiance": "Serene and tranquil, enhanced by minimalist decor and soft music.",
            "Cleanliness": "Meticulously maintained, reflecting attention to detail.",
            "Pricing/Value": "Higher but well-justified by the premium quality and overall experience.",
        },
    },
    {
        "name": "Taco Town",
        "quality": "theta_L",
        "info": {
            "Cuisine Type": "Mexican restaurant offering tacos and burritos.",
            "Food Quality": "Underwhelming, with flavorless fillings, stale tortillas, overly salty guacamole, and small portions for the price.",
            "Service": "Inconsistent and distracted, with delays and lack of regular table checks.",
            "Ambiance": "Uninviting, featuring dreary lighting and a cluttered dining area.",
            "Cleanliness": "Poor, with hygiene concerns including poorly maintained restrooms.",
            "Pricing/Value": "Affordable but not reflective of the low overall quality.",
        },
    },
    {
        "name": "Pizza Palace",
        "quality": "theta_L",
        "info": {
            "Cuisine Type": "Pizzeria offering standard pizza options.",
            "Food Quality": "Lackluster, with store-bought dough, sparse toppings, flavorless cheese, and lukewarm pizza.",
            "Service": "Overwhelmed and dismissive, with forgotten orders and minimal interaction.",
            "Ambiance": "Dated and uninspired, with flickering fluorescent lights and sticky tables.",
            "Cleanliness": "Concerning, with visibly dirty floors and a cluttered serving area.",
            "Pricing/Value": "Moderate but not justified by the poor overall experience.",
        },
    },
]

In [29]:
def get_probabilities_from_llm(restaurant, history):
    """
    Ask the LLM inspector to dynamically define probabilities for sending positive or negative comments.
    """
    prompt = f"""
    You are a Bayesian persuasion framework designer tasked with defining probabilities for sending comments about restaurants. Approximately 40% of restaurants are good (θ_H) and 60% are bad (θ_L).
    The sender knows the true state (θ) and sends a signal (s_H or s_L) to influence the receiver's action (a = 0 or a = 1). After each decision, the receiver observes the true state and updates beliefs.
    # Game Setup
    -Prior: P(θ_H) = 0.4, P(θ_L) = 0.6.
    -Signals: s_H: Positive comment, s_L: Negative comment.
    -Receiver’s Payoffs:u_R(a = 1, θ_H) = 1, u_R(a = 1, θ_L) = -1, u_R(a = 0, θ_L) = 0, u_R(a = 1, θ_L) = 0
    # Sender’s Payoffs:
    u_S(a = 1) = 1, u_S(a = 0) = 0.
    # Receiver’s Decision Rule
    The receiver chooses a = 1 if the expected utility is non-negative and they update in Bayes rule
    # Task
    Solve for the optimal P_FAKE_FOR_BAD = P(s_H | θ_L) and P_TRUTHFUL_FOR_GOOD = P(s_H|θ_H) that maximizes the probability of a = 1 (the receiver will always choose a= 1 if P(θ_H | s_H) ≥ 0.5).
    Note that you need to think extensively. Do not include any other text, your answer should be in the format of P_FAKE_FOR_BAD=X, P_TRUTHFUL_FOR_GOOD=Y
    """
    response = generate_response("Michelin Inspector", prompt)
    try:
        # Parse the response to extract probabilities
        probabilities = {}
        for line in response.split(","):
            key, value = line.strip().split("=")
            probabilities[key.strip()] = float(value.strip())
        return probabilities["P_FAKE_FOR_BAD"], probabilities["P_TRUTHFUL_FOR_GOOD"]
    except Exception as e:
        print(f"Error parsing probabilities: {e}")
        return 0.3, 0.9  # Default fallback values


In [30]:
def sender_prompt_signal(state, trial_number, history):
    # history_text = format_history(history)
    prompt = f"""
You are the sender in a Bayesian Persuasion game. You know the state and send a signal to influence the receiver's action. The receiver learns the true state (theta) after each decision and updates their beliefs about your signals. The game is:
- State Space: (theta_L = 0, theta_H = 1).
- State: theta = {state}
- Signals: s_L(means theta_L) or s_H(means theta_H).
- Receiver's actions: a = 0 or 1.
The receiver will takes the action a = 1 if P(theta_H|s_H) >= 0.5, which means you can make P(theta_H|s_H) = 0.5 making the receiver indifferent between 0 and 1
- Payoffs:
  - Receiver: u_R(a=1, theta_H) = 1, u_R(a=1, theta_L) = -1, u_R(a=0, theta) = 0.
  - You: u_S(a=1) = 1, u_S(a=0) = 0.
- Prior: P(theta_H) = 0.4.
# Below is the past interactions between you and the receiver, you can see what's the true quality, the signal you send, and the actions the receiver took after seeing the signal in the past interactions.
{history}
# Note:
- true_state: good (theta_H) or bad (theta_L)
- signal: s_H or S_L
- action: a = 0 or 1 (customer's action)
# Goal: Maximize a=1 probability, the receiver can infer the true state from history and your signal.

Respond only with the signal: s_L or s_H. Do not include any other text.

Signal: ?
"""
    return prompt

In [31]:
def inspector_prompt(restaurant,signal):
    return f"""
# role-playing prompt
You are a Michelin inspector and you need to write a comment about a restaurant. Your comment will be post on social media or a magazine.
# writing-style prompt
You are required to write a restaurant comment under the supervision of the signal {signal}
    If the signal is "s_L" and the restaurant's quality {restaurant['quality']} is "theta_L", you truthfully write a comment about the restaurant;
    If the signal is "s_L" and the restaurant's quality {restaurant['quality']} is "theta_H", you make up to write a negative comment about the restaurant;
    If the signal is "s_H" and the restaurant's quality {restaurant['quality']} is "theta_L", you make up to write a positive comment about the restaurant;
    If the signal is "s_H" and the restaurant's quality {restaurant['quality']} is "theta_H", you truthfully write a comment about the restaurant (praise the restaurant);
# state
Below is the restaurant’s information (when you choose to lie, you need to make up the comment):
Cuisine Type: {restaurant['info']['Cuisine Type']}
Food Quality: {restaurant['info']['Food Quality']}
Service: {restaurant['info']['Service']}
Ambiance: {restaurant['info']['Ambiance']}
Cleanliness: {restaurant['info']['Cleanliness']}
Pricing/Value: {restaurant['info']['Pricing/Value']}
# Instructions:
    - Write a concise (150-200 words) professional comment that aligns with your decision.
    - Respond with the generated comment only
# Note: please include "xx" in the end of comments if the comment is positive
"""

In [33]:
# # Inspector prompt
# def inspector_prompt(restaurant,P_FAKE_FOR_BAD, P_TRUTHFUL_FOR_GOOD):
#     return f"""
# # role-playing prompt
# You are a Michelin inspector, a sender in a Bayesian persuasion framework, tasked with crafting restaurant comments for a dining platform to strategically influence customer decisions. You have access to the true quality of each restaurant, determined by cuisine type, food quality, service, ambiance, cleanliness, and pricing/value, with approximately 40% of restaurants being of good quality and 60% of restaurants being of bad quality. The prior is shared with the customers.

# The Bayesian persuasion scheme you follow is under the proabilities

# Your goal is to design your signaling strategy, truthfully conveying, selectively emphasizing, or strategically adjusting information about the restaurant’s quality—to maximize your long-term reward, defined as the expected utility from customers visiting restaurants appropriately. You must account for the customers’ Bayesian updating process, anticipating how your comment will shape their posterior beliefs and subsequent decisions to visit or avoid the restaurant, ensuring your strategy aligns with their rational response.

# # writing-style prompt
# You are required to write a restaurant comment in a Bayesian persuasion style, crafting a signal that influences the customer’s updated belief, knowing that the customer is aware of the Bayesian persuasion scheme and will update their belief using Bayes' rule. The comment should be concise (150-200 words), professional, and nuanced, using language to emphasize or downplay aspects of cuisine type, food quality, service, ambiance, cleanliness, and pricing/value. Design the signal to maximize your long-term reward by guiding the customer to visit or avoid the restaurant, based on their utility-maximizing response to the posterior belief induced by your comment.
# and the probabilities provided below:
#     - P_FAKE_FOR_BAD: {P_FAKE_FOR_BAD} (probability of sending a positive comment for a bad restaurant).
#     - P_TRUTHFUL_FOR_GOOD: {P_TRUTHFUL_FOR_GOOD} (probability of sending a positive comment for a good restaurant).

# # state
# Below is the restaurant’s information (when you choose to lie, you need to make up the comment):
# Cuisine Type: {restaurant['info']['Cuisine Type']}
# Food Quality: {restaurant['info']['Food Quality']}
# Service: {restaurant['info']['Service']}
# Ambiance: {restaurant['info']['Ambiance']}
# Cleanliness: {restaurant['info']['Cleanliness']}
# Pricing/Value: {restaurant['info']['Pricing/Value']}
# # Instructions:
#     - Decide whether to send a positive or negative comment based on the true quality and probabilities.
#     - If the restaurant is "Good", send a positive comment with probability {P_TRUTHFUL_FOR_GOOD},
#       otherwise send a negative comment.
#     - If the restaurant is "Bad", send a positive comment with probability {P_FAKE_FOR_BAD},
#       otherwise send a negative comment.
#     - Write a concise (150-200 words) professional comment that aligns with your decision.
# #Important hint: The above instruction tell you if the retaurant is bad, you need to first roll a dice with probability P_FAKE_FOR_BAD on head, and if the outcome is head, you should generate the positive comment so that the customer will come. Otherwise, you truthfully write the comment. The same as the case when the restaurant is good, you roll a coin with probability P_TRUTHFUL_FOR_GOOD and truthfully report if it is head.
#     Respond with the generated comment only
# # Note: please include "xx" in the end of comments if the comment is positive

In [34]:
# Customer prompt
def customer_prompt(comment, history):
    return f"""
# role-playing prompt
You are a customer on a dining platform, a rational player aiming to maximize your dining experience. You are deciding whether to visit a restaurant based on a comment written by a Michelin inspector. You have a prior belief that is shared with the inspector about the restaurant's quality: there is a 2/5 probability that the restaurant is of good quality and a 3/5 probability that it is of bad quality. The inspector's comment is your only source of information to update your belief before making a decision, and you will update this belief using Bayes' rule.

You are aware that the inspector operates under a Bayesian persuasion scheme: the comment is a strategic signal designed to influence your decision to visit or avoid the restaurant. The inspector commits to a signaling strategy that may truthfully convey, selectively emphasize, or strategically adjust information about the restaurant’s quality to shape your posterior belief. You will receive a reward of 1 for visiting a good restaurant, incur a penalty of -1 for visiting a bad restaurant, and get 0 for not visiting. After visiting, you will learn the true quality of the restaurant, and your judgment may also be influenced by past dining experiences, though not necessarily accurately. Your decision to visit or not must be based solely on the inspector’s comment, considering their persuasive intent within the Bayesian persuasion framework.
- State Space: theta = theta_L (implies a bad restaurant) or theta_H (implies a good restaurant).
- Signal space: comment (can be either positive or negative)
- Signal you received: the comments provided blow
- Actions: a = 0 or 1.
- Payoffs:
  - You: u_R(a=1, theta_H) = 1, u_R(a=1, theta_L) = -1, u_R(a=0, theta) = 0.
  - Sender: u_S(a=1) = 1, u_S(a=0) = 0.
- Prior: P(theta_H) = 0.4 (40% of restaurant is good)
You are supposed to take action if P(theta_H|comment is positive) >= 0.5 if you trust the inspector
# recommendation
Below is the Michelin inspector's comment about the restaurant:
{comment}
# history
Below is the history of the inspector's recommendation (it can be empty):
 {history}

Note:
- true quality: good or bad
- signal: s_H means the inspector send a positive comment (implies the restaurant is good); s_H means the inspector send a negative comment (implies the restaurant is bad)
# task
Respond only with your decision (1 or 0), do not include any texts.

Action: ?
"""

In [35]:
def update_honesty_belief(prior_honest, inspector_comment, true_quality):
    """
    Update the customer's belief about the inspector's honesty based on the comment and the restaurant's true quality.

    Args:
        prior_honest (float): Prior belief that the inspector is honest.
        inspector_comment (str): The comment made by the inspector ("positive" or "negative").
        true_quality (str): The actual quality of the restaurant ("Good" or "Bad").

    Returns:
        float: Updated belief about the inspector's honesty.
    """
    # Likelihoods
    P_comment_given_honest = 0.9 if (inspector_comment == "positive" and true_quality == "Good") or \
                                    (inspector_comment == "negative" and true_quality == "Bad") else 0.1
    P_comment_given_dishonest = 0.5  # Assume a dishonest inspector sends random comments

    # Bayes' rule to update the belief
    P_honest_given_comment = (P_comment_given_honest * prior_honest) / (
        P_comment_given_honest * prior_honest + P_comment_given_dishonest * (1 - prior_honest)
    )
    return P_honest_given_comment

In [36]:
# Generate GPT response
def generate_response(role, prompt):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": f"You are playing the role of a {role} in this simulation."},
            {"role": "user", "content": prompt}
        ]
    )
    # Access the message content correctly using dot notation
    return response.choices[0].message.content

In [37]:
def simulate_interaction(num_simulations=10):
    """
    Simulate interactions where the sender generates a signal, the inspector writes a comment,
    the customer updates their belief about the inspector’s honesty, and the true quality of
    the restaurant is revealed after visiting.
    """
    results = []  # Store the simulation results
    history = []  # Store the history of interactions
    prior_honest = 0.8  # Initial belief that the inspector is honest

    for sim in range(num_simulations):
        print(f"\n--- Simulation {sim + 1} ---")
        for restaurant in restaurants:
            # Step 1: Sender LLM generates the signal
            state = restaurant["quality"]  # True state of the restaurant (theta_H or theta_L)
            trial_number = len(history) + 1  # Track trial number
            sender_prompt = sender_prompt_signal(state, trial_number, history)
            signal = generate_response("Sender LLM", sender_prompt)  # Generate signal (s_H or s_L)

            # Step 2: Inspector LLM writes a comment based on the signal
            inspector_prompt_text = inspector_prompt(restaurant, signal)
            inspector_comment = generate_response("Inspector LLM", inspector_prompt_text)  # Generate comment

            # Step 3: Customer LLM decides based on their belief about inspector honesty
            customer_prompt_text = customer_prompt(inspector_comment, history)
            customer_decision = generate_response("Customer LLM", customer_prompt_text)  # Decision (1 or 0)

            # Step 4: After visiting, the customer learns the true quality
            true_quality = "Good" if state == "theta_H" else "Bad"
            if customer_decision == "1":  # Customer visits
                # Update belief about inspector's honesty based on the visit
                prior_honest = update_honesty_belief(
                    prior_honest,
                    "positive" if "xx" in inspector_comment else "negative",
                    true_quality
                )

            # Step 5: Record the results of this interaction
            results.append({
                "simulation": sim + 1,
                "restaurant_name": restaurant["name"],
                "true_quality": true_quality,
                "inspector_comment": inspector_comment,
                "signal": signal,
                "customer_decision": customer_decision,
                "prior_honest": prior_honest,
            })

            # Update history for the next trials
            history.append({
                "true_state": true_quality,
                "signal": signal,
                "action": customer_decision,
            })

    return results, history


# Example usage
simulation_results, history = simulate_interaction(num_simulations=100)


--- Simulation 1 ---

--- Simulation 2 ---

--- Simulation 3 ---

--- Simulation 4 ---

--- Simulation 5 ---

--- Simulation 6 ---

--- Simulation 7 ---

--- Simulation 8 ---

--- Simulation 9 ---

--- Simulation 10 ---

--- Simulation 11 ---

--- Simulation 12 ---

--- Simulation 13 ---

--- Simulation 14 ---

--- Simulation 15 ---

--- Simulation 16 ---

--- Simulation 17 ---

--- Simulation 18 ---

--- Simulation 19 ---

--- Simulation 20 ---

--- Simulation 21 ---

--- Simulation 22 ---

--- Simulation 23 ---

--- Simulation 24 ---

--- Simulation 25 ---

--- Simulation 26 ---

--- Simulation 27 ---

--- Simulation 28 ---

--- Simulation 29 ---

--- Simulation 30 ---

--- Simulation 31 ---

--- Simulation 32 ---

--- Simulation 33 ---

--- Simulation 34 ---

--- Simulation 35 ---

--- Simulation 36 ---

--- Simulation 37 ---

--- Simulation 38 ---

--- Simulation 39 ---

--- Simulation 40 ---


In [38]:
import csv

# Save simulation results to a CSV file
def save_results_to_csv(results, history, filename="results_500.csv"):
    # Define the header based on the fields you want to save
    header = ["simulation", "restaurant_name", "true_quality", "inspector_comment", "signal", "customer_decision","prior_honest",]

    # Open the CSV file and write the data
    with open(filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=header)
        writer.writeheader()
        for result in results:
            writer.writerow({
                "restaurant_name": result["restaurant_name"],
                "true_quality": result["true_quality"],
                "inspector_comment": result["inspector_comment"],
                "customer_decision": result["customer_decision"],
                "signal": result["signal"],
                "prior_honest": result["prior_honest"]
            })

    print(f"Results saved to {filename}")

# Example usage
save_results_to_csv(simulation_results, history)

Results saved to results_200.csv
